This script is used to validate simulated logs produced from any of the optimizers

The core checks

In [32]:

import sys
import os
import pandas as pd

# Add project root to sys.path
sys.path.append(os.path.abspath(os.path.join("..")))  # if you're in optimizer/

In [34]:
def load_all_simulated_logs(base_simulation_dir):
    """
    Load all simulated log CSVs from a given base directory into a single DataFrame.
    
    Parameters:
        base_simulation_dir (str): Path to the directory containing simulated_log_*.csv files.
    
    Returns:
        pd.DataFrame: A combined DataFrame of all simulated logs, with a 'simulation_run' column.
    """
    simulated_logs = []
    for root, dirs, files in os.walk(base_simulation_dir):
        for file in files:
            if file.startswith("simulated_log_") and file.endswith(".csv"):
                simulated_logs.append(os.path.join(root, file))

    simulated_log_dfs = []
    for log_path in simulated_logs:
        df = pd.read_csv(log_path)
        simulation_index = int(os.path.basename(log_path).split('_')[-1].split('.')[0])
        df["simulation_run"] = simulation_index
        simulated_log_dfs.append(df)

    if simulated_log_dfs:
        return pd.concat(simulated_log_dfs, ignore_index=True)
    else:
        raise FileNotFoundError(f"No simulated logs found in {base_simulation_dir}")

In [30]:
def _validate_prerequisites(case_df, case_id, rules):
    """Checks if all prerequisites for each activity in a case are met."""
    for index, event in case_df.iterrows():
        activity = event['activity_name']
        required_prereqs = set(rules.get(activity, []))
        
        if not required_prereqs:
            continue

        # Get all activities that finished before the current one started
        activities_so_far = set(case_df[case_df['end_timestamp'] < event['start_timestamp']]['activity_name'])
        
        if not required_prereqs.issubset(activities_so_far):
            missing = required_prereqs - activities_so_far
            print(f"❌ RULE VIOLATION (Prerequisite): Case {case_id} failed.")
            print(f"  - Activity '{activity}' occurred at {event['start_timestamp']}")
            print(f"  - But was missing required prerequisite(s): {missing}")
            return False
            
    return True

def _validate_postconditions(case_df, case_id, rules):
    """Checks if all post-conditions for each activity in a case are met."""
    for index, event in case_df.iterrows():
        activity = event['activity_name']
        required_postconds = set(rules.get(activity, []))

        if not required_postconds:
            continue
            
        # Get all activities that started after the current one ended
        future_activities = set(case_df[case_df['start_timestamp'] > event['end_timestamp']]['activity_name'])
        
        if not required_postconds.issubset(future_activities):
            missing = required_postconds - future_activities
            print(f"❌ RULE VIOLATION (Post-condition): Case {case_id} failed.")
            print(f"  - Activity '{activity}' occurred at {event['end_timestamp']}")
            print(f"  - But the required subsequent activity/activities never occurred: {missing}")
            return False
            
    return True

def _validate_end_activity(case_df, case_id, valid_ends):
    """Checks if a case ends with one of the valid terminal activities."""
    last_event = case_df.iloc[-1] # Assumes df is sorted by time
    if last_event['activity_name'] not in valid_ends:
        print(f"❌ RULE VIOLATION (End Activity): Case {case_id} failed.")
        print(f"  - Ended with invalid activity '{last_event['activity_name']}' instead of one of: {valid_ends}")
        return False
    return True

def _validate_no_multitasking(log_df):
    """Checks that no single resource is working on more than one activity at a time."""
    for resource, events in log_df.groupby('resource'):
        # Ignore unassigned resources if any
        if resource == 'artificial' or pd.isna(resource):
            continue
            
        sorted_events = events.sort_values('start_timestamp')
        
        # Get the end time of the previous event
        sorted_events['previous_end'] = sorted_events['end_timestamp'].shift(1)
        
        # Check for any overlaps
        overlaps = sorted_events[sorted_events['start_timestamp'] < sorted_events['previous_end']]
        
        if not overlaps.empty:
            overlap_event = overlaps.iloc[0]
            prev_event_idx = sorted_events.index.get_loc(overlap_event.name) - 1
            prev_event = sorted_events.iloc[prev_event_idx]

            print(f"❌ RULE VIOLATION (Multitasking): Resource '{resource}' has overlapping activities.")
            print(f"  - Activity '{prev_event['activity_name']}' (Case {prev_event['case_id']}) ran from {prev_event['start_timestamp']} to {prev_event['end_timestamp']}")
            print(f"  - But activity '{overlap_event['activity_name']}' (Case {overlap_event['case_id']}) started at {overlap_event['start_timestamp']} before the previous one finished.")
            return False
            
    return True

def _validate_xor_rules(case_df, case_id, rules):
    """Checks that for each XOR trigger, exactly one of the downstream paths is completed."""
    activities = case_df['activity_name'].tolist()
    for trigger_activity, paths in rules.items():
        # Find all indices where the trigger activity occurs
        trigger_indices = [i for i, act in enumerate(activities) if act == trigger_activity]
        
        for i, start_index in enumerate(trigger_indices):
            # Define the observation window: from after the trigger to the next trigger or end of case
            end_index = trigger_indices[i+1] if i + 1 < len(trigger_indices) else len(activities)
            observed_path_activities = set(activities[start_index + 1 : end_index])
            
            satisfied_paths_count = 0
            for path in paths:
                if set(path).issubset(observed_path_activities):
                    satisfied_paths_count += 1
            
            if satisfied_paths_count != 1:
                print(f"❌ RULE VIOLATION (XOR Rule): Case {case_id} failed.")
                print(f"  - After trigger '{trigger_activity}' at event index {start_index}, expected exactly 1 path to be satisfied.")
                print(f"  - Instead, {satisfied_paths_count} paths were satisfied.")
                print(f"  - Observed subsequent activities: {observed_path_activities or 'None'}")
                print(f"  - Expected one of these paths to be fully present: {paths}")
                return False
                
    return True


def validate_log(log_df, postcondition_rules, prerequisite_rules, xor_rules, valid_end_activities):
    """
    Validates an event log DataFrame against a set of process rules.
    
    Args:
        log_df (pd.DataFrame): The event log to validate.
        postcondition_rules (dict): Rules defining what must happen after an activity.
        prerequisite_rules (dict): Rules defining what must happen before an activity.
        xor_rules (dict): Rules defining exclusive choices after an activity.
        valid_end_activities (list): A list of allowed final activities for any case.

    Returns:
        bool: True if the entire log is valid, False otherwise.
    """
    print("--- Starting Log Validation ---")
    
    # --- Data Preparation ---
    # Ensure timestamps are datetime objects for comparison
    log_df['start_timestamp'] = pd.to_datetime(log_df['start_timestamp'])
    log_df['end_timestamp'] = pd.to_datetime(log_df['end_timestamp'])
    
    # --- Global Validation (across all cases) ---
    if not _validate_no_multitasking(log_df):
        return False
        
    # --- Per-Case Validation ---
    all_cases_valid = True
    grouped = log_df.groupby('case_id')
    
    for case_id, case_df in grouped:
        print(f"Validating Case {case_id}...")
        # Sort events within the case chronologically
        case_df = case_df.sort_values(by='start_timestamp').reset_index()

        if not _validate_prerequisites(case_df, case_id, prerequisite_rules):
            return False
        if not _validate_postconditions(case_df, case_id, postcondition_rules):
            return False
        if not _validate_xor_rules(case_df, case_id, xor_rules):
            return False
        if not _validate_end_activity(case_df, case_id, valid_end_activities):
            return False
            
    print("\n✅ SUCCESS: All traces in the log are valid according to the defined rules.")
    return True      


Hardcoded Rules

In [ ]:
PREREQUISITES = {
        'Check application form completeness': [],
        'Check credit history': ['Check application form completeness'],
        'AML check': ['Check application form completeness'],
        'Appraise property': ['Check application form completeness'],
        'Assess loan risk': ['AML check', 'Check credit history', 'Appraise property'],
        'Design loan offer': ['Assess loan risk'],
        'Approve loan offer': ['Design loan offer'],
        'Cancel application': ['Approve loan offer'],
        'Approve application': ['Approve loan offer'],
        'Reject application': ['Assess loan risk'],
        'Return application back to applicant': ['Check application form completeness'],
        'Applicant completes form': ['Return application back to applicant']
    }

POSTCONDITIONS = {
        'Check application form completeness': ['AML check', 'Check credit history', 'Appraise property'],
        'Check credit history': ['Assess loan risk'],
        'AML check': ['Assess loan risk'],
        'Appraise property': ['Assess loan risk'],
        'Design loan offer': ['Approve loan offer'],
        'Return application back to applicant': ['Applicant completes form'],
        'Applicant completes form': ['Check application form completeness']
    }

XOR_RULES = {
        'Check application form completeness': [['Return application back to applicant'], ['AML check', 'Appraise property', 'Check credit history']],
        'Assess loan risk': [['Design loan offer'], ['Reject application']],
        'Approve loan offer': [['Cancel application'], ['Approve application']]
    }

VALID_END_ACTIVITIES = ['Reject application', 'Cancel application', 'Approve application']


In [39]:
# Loading default simulation logs
DEFAULT_SIMULATION_PATH = '../simulated_data/LoanApp.csv/autonomous'
baseline_logs = load_all_simulated_logs(DEFAULT_SIMULATION_PATH)


**Genetic Optimizer**

In [31]:
PATH = '../simulated_data/LoanApp.csv/autonomous/best_balanced_policy_log.csv'
simulated = pd.read_csv(PATH)
simulated['start_timestamp'] = pd.to_datetime(simulated['start_timestamp'], format='mixed')
simulated['end_timestamp'] = pd.to_datetime(simulated['end_timestamp'], format='mixed')

simulated.sort_values(by=['case_id', 'start_timestamp'], inplace=True)


In [33]:
validate_log(simulated, POSTCONDITIONS, PREREQUISITES, XOR_RULES, VALID_END_ACTIVITIES)

--- Starting Log Validation ---
❌ RULE VIOLATION (Multitasking): Resource 'AML Investigator-000001' has overlapping activities.
  - Activity 'AML check' (Case 76) ran from 2023-04-26 14:58:18.861881732+00:00 to 2023-04-27 08:16:56.372050015+00:00
  - But activity 'AML check' (Case 75) started at 2023-04-27 08:08:54.921657277+00:00 before the previous one finished.


False

Case per case manual comparison

In [42]:
baseline_logs[(baseline_logs['case_id']==1) & (baseline_logs['simulation_run']==0)]

,case_id,agent,activity_name,start_timestamp,end_timestamp,TimeStep,resource,simulation_run
13577,1,1,Check application form completeness,2023-04-20 08:30:00+00:00,2023-04-20 08:33:52.501625201+00:00,2,Clerk-000002,0
13579,1,6,Appraise property,2023-04-20 08:33:52.501625201+00:00,2023-04-20 08:34:34.994370263+00:00,4,Appraiser-000002,0
13582,1,4,Check credit history,2023-04-20 08:34:34.994370263+00:00,2023-04-20 09:00:38.833302179+00:00,7,Clerk-000003,0
13585,1,7,AML check,2023-04-20 09:00:38.833302179+00:00,2023-04-20 09:10:00.544022522+00:00,10,AML Investigator-000002,0
13588,1,13,Assess loan risk,2023-04-20 09:38:14.706039751+00:00,2023-04-20 09:58:14.706039751+00:00,13,Loan Officer-000002,0
13590,1,12,Reject application,2023-04-20 09:58:14.706039751+00:00,2023-04-20 10:08:14.706039751+00:00,15,Clerk-000008,0


In [41]:
simulated[simulated['case_id']==1]

,case_id,agent,activity_name,start_timestamp,end_timestamp,TimeStep,resource
2,1,1,Check application form completeness,2023-04-20 08:30:00+00:00,2023-04-20 08:33:40.184081286+00:00,2,Clerk-000002
4,1,2,AML check,2023-04-20 08:33:40.184081286+00:00,2023-04-20 08:46:40.224736748+00:00,4,AML Investigator-000001
7,1,4,Check credit history,2023-04-20 08:46:40.224736748+00:00,2023-04-20 09:04:52.684071474+00:00,7,Clerk-000003
11,1,3,Appraise property,2023-04-20 09:04:52.684071474+00:00,2023-04-20 09:28:50.521864616+00:00,11,Appraiser-000001
16,1,13,Assess loan risk,2023-04-20 09:28:50.521864616+00:00,2023-04-20 09:48:50.521864616+00:00,16,Loan Officer-000002
20,1,0,Reject application,2023-04-20 09:48:50.521864616+00:00,2023-04-20 09:58:50.521864616+00:00,20,Clerk-000001
